In [1]:
import geopandas as gp
import glob
import pandas as pd
import numpy as np
from datetime import datetime

shp_file_ = glob.glob("./tl_2016_us_county/*.shp")
csv_file_ = glob.glob("*.csv")
print("SHP:", shp_file_)
print("CSV:", csv_file_)
gdf = gp.read_file(shp_file_[0])
florida_geo = gdf[gdf.STATEFP.astype(int) == 12]

# read COVID-19 cases data by date
all_state = pd.read_csv(csv_file_[0])
#fill na with 0
all_state = all_state.fillna(0)

SHP: ['./tl_2016_us_county/tl_2016_us_county.shp']
CSV: ['us-counties.csv', 'time_series_covid19_confirmed_global.csv', 'us-states.csv']


In [19]:
florida_df = all_state[all_state.state == "Florida"]
#get ips for different counties
fips_ = florida_df.fips.unique()
#get dates in date_list for plotting
date_list = florida_df.date.unique()
temp_01 = florida_df.groupby("fips").get_group(12057.0).T
#Formatting the dataframe
converted_list = []
for fips in fips_:
    temp_01 = florida_df.groupby('fips').get_group(fips).T
    temp_01.columns = temp_01.loc['date']
    temp_01 = temp_01.drop(['date', 'county', 'state', 'fips', 'deaths'], axis = 0)
    temp_01.insert(0,"GEOID",fips)
    converted_list.append(temp_01)
#Convert to the final df
case_df = pd.concat(converted_list, axis = 0)
#Fill the na
case_df = case_df.fillna(0)

In [23]:
plot_df = florida_df.groupby("fips").get_group(12001.0)

In [39]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x= plot_df.date, y=plot_df.cases,
                    mode='lines+markers',
                    name='lines',
                    marker=dict(color='LightSkyBlue', size=20,
                                line=dict(color='MediumPurple', width=2)
                               ),
                    showlegend=False
                        )
             )
fig.show()